# AMS 520 Final Project

This notebook contains codes and report implementing 'Model high-frequency limit order book dynamics with SVM' written by Edward Cummings, Anton Malandii, Jack Peters, and Weiwei Tao.

In [161]:
!pip install -qq pyarrow
!pip install -qq xgboost

In [162]:
import functools as fn
import datetime
import math

import pandas as pd
import numpy as np
import scipy.stats as scs

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import DecisionBoundaryDisplay

import seaborn as sns
import matplotlib.pyplot as plt


sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

## Part 1. Data Exploration

### Trade dataset

The trade dataset contains 243K UTP trade records from 14 different Stock Exchanges on 2022-01-06 between 4 a.m. until 8 p.m. EST for AAPL.


### Quote dataset
The quote dataset contains 473K UTP quote records from 13 different Stock Exchanges on 2022-01-06 for AAPL.

In [163]:
t = pd.read_csv('query_trades_AAPL_20200106_born202210121745.csv',)
q = pd.read_csv('query_quotes_AAPL_20200106_born202210121744.csv',)

/var/folders/9g/k3f7lx5n4cnb1yd775c6k7bw0000gn/T/ipykernel_34631/2188346647.py:2: DtypeWarning:

Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.



In [164]:
t.shape, q.shape

((242878, 17), (473340, 31))

## Part 2. Data Cleaning

We first removed the closing/opening auction dissemination ( remove first and last 15 min) and excluded trades from the exchange D from our analysis. Then, trades and quote updates that stem from the same marketable order execution were regrouped together. We used the participant timestamp to assign a unique marketable order execution (MOX) identifier to each batch of trades and quotes triggered in execution of a single marketable order. Note that the reason that the participant timestamp can be used to derive the MOX identifier because all trades and quote updates that are triggered in the execution of the same marketable order receive the same participant (but not SIP) timestamp in high resolution.

After data cleaning, the analysis dataset contains 395K records. For each records, following features are generated:
1. Time
2. ask/bid price/volume, mid price, spread
3. dPask_dt, dPbid_dt, dVask_dt, dVbid_dt
4. average trade price/volume within 10ms, 100ms, 1s and 10s. 

In [165]:
from taq_data_cleaning import gen_basic_features_TAQ, gen_targets_events

In [166]:
df_features = gen_basic_features_TAQ(t, q,)

print(df_features.shape)
df_features.columns

/Users/weiweitao/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning:

In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)



(395698, 20)


Index(['index', 'time', 'ask_price', 'bid_price', 'ask_volume', 'bid_volume',
       'mid_price', 'spread', 'dPask_dt', 'dPbid_dt', 'dVask_dt', 'dVbid_dt',
       'avg_trade_price_10ms', 'avg_trade_price_100ms', 'avg_trade_price_1s',
       'avg_trade_price_10s', 'avg_trade_volume_10ms',
       'avg_trade_volume_100ms', 'avg_trade_volume_1s',
       'avg_trade_volume_10s'],
      dtype='object')

## Part 3. Generate Labels

Both event-base and time-based outcomes were evaluated. 
1. After 30 orders, whether the price and spread will go down, unchanged or up.
2. After 100 ms, whether the price and spread will go down, unchanged or up.

In [167]:
df_events = gen_targets_events(q,num_events=30)
df_events.head()

,index,id,time,midup,middown,mideq,spreadup,spreaddown,spreadeq
0,42179,2214126,35100.044580,False,True,False,False,False,True
1,42181,2214141,35100.044763,False,True,False,False,False,True
2,42182,2214146,35100.044834,False,True,False,False,False,True
3,42183,2214167,35100.045857,False,True,False,False,False,True
4,42184,2214865,35100.094862,False,True,False,False,False,True


In [168]:
df_events['spreadup'].value_counts()

False    369298
True      26370
Name: spreadup, dtype: int64

In [169]:
df_events['spreaddown'].value_counts()

False    363617
True      32051
Name: spreaddown, dtype: int64

In [170]:
df_events['spreadeq'].value_counts()

True     363617
False     32051
Name: spreadeq, dtype: int64

In [172]:
df_events.loc[df_events['spreadup'] == True, 'yspread'] = 'Up'
df_events.loc[df_events['spreadeq'] == True, 'yspread'] = 'Equal'
df_events.loc[df_events['spreaddown'] == True, 'yspread'] = 'Down'
df_events['yspread'].value_counts()

Equal    363617
Down      32051
Name: yspread, dtype: int64

In [ ]:
df_targets_time = gen_targets_temporal(q,prediction_interval=100)
df_targets_time.sample(8)

In [ ]:
def create_df_combined(df_features, df_targets_time, df_events=None ):

    #Explicit merge, should use left join instead
    try:
        df_features.set_index('id', inplace=True)
    except:
        pass
    
    df_features.sort_index(inplace=True)
    try:
        df_targets_time.set_index('id', inplace=True) 
    except:
        pass
    
    df_targets_time.sort_index(inplace=True)
    ntarg = []
    for col in df_targets_time.columns:
        if 'mid' in col or 'spread' in col:
            ntarg += [col]
            
    ind_comb = set(df_features.index.to_numpy()).intersection(set(df_targets_time.index.to_numpy()))
    df_combine = df_features.loc[ind_comb]
    df_combine[ntarg] = df_targets_time[ntarg].loc[ind_comb]
    
    if ~isinstance(df_events, type(None)):
        try:
            df_events.set_index('id', inplace=True)
        except:
            pass
        nevent = []
        for col in df_events.columns:
            if 'mid' in col or 'spread' in col:
                nevent += [col]
        ind_comb = ind_comb.intersection(set(df_events.index.to_numpy()))
   
        df_combine[nevent] = df_events[nevent].loc[ind_comb]
    return df_combine

In [ ]:
df_combine =  create_df_combined(df_features, df_targets_time, df_events)

In [ ]:
df_combine.columns

In [ ]:
df_combine.reset_index(inplace=True)
df_combine.to_feather("clean_combine_30e_50ms_AAPL_20200106.f") ## checkpoint combined features/targets

In [ ]:
df_combine.head()

In [ ]:
import plotly.graph_objects as go
cols = []
for col in df_combine.columns:
    if 'mid' in col or 'spread' in col:
        cols += [col]
cols = cols[2:]
fig = go.Figure(layout=go.Layout(height=400, width=800))
fig.add_bar(x=cols,y=df_combine[cols].sum()/len(df_combine))
fig.update_layout(title_text="Distribution of Outcomes")
fig.show()

In [ ]:
df_combine.rename(columns = {'avg_trade_price_10ms':'AvgP_10ms', 
                             'avg_trade_price_100ms':'AvgP_100ms', 
                             'avg_trade_price_1s':'AvgP_1s', 
                             'avg_trade_price_10s':'AvgP_10s', 
                             'avg_trade_volume_10ms':'AvgV_10ms', 
                             'avg_trade_volume_100ms':'AvgV_100ms', 
                             'avg_trade_volume_1s':'AvgV_1s', 
                             'avg_trade_volume_10s':'AvgV_10s', 
                            }, inplace = True)

focus_columns = ['ask_price', 'bid_price', 'ask_volume', 'bid_volume', 
                 'mid_price', 'spread', 'dPask_dt', 'dPbid_dt', 'dVask_dt', 'dVbid_dt', 
                 'AvgP_10ms', 'AvgP_100ms', 'AvgP_1s', 'AvgP_10s', 
                 'AvgV_10ms', 'AvgV_100ms', 'AvgV_1s', 'AvgV_10s']

corr=df_combine[focus_columns].corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

heat_fig, (ax)=plt.subplots(1, 1, figsize=(8,5))

heat=sns.heatmap(corr, 
                   ax=ax,  
                   mask=mask, vmin=-1, vmax=1, annot=False, cmap='BrBG')

heat_fig.subplots_adjust(top=.93)

In [ ]:
df_features['hour'] = df_features['time']/3600
df_features.set_index('hour', inplace = True)
cols = ['ask_price', 'bid_price', 'ask_volume', 'bid_volume', 'mid_price', 'spread']

import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000


plt.style.use('fivethirtyeight')
df_features[cols].plot(subplots=True, figsize=(12, 9),fontsize=10, sharex=False, layout=(3, 2),  linewidth=2, title='Visualization of the original Time Series');

## Part 4. Train-Test Splitting

In [ ]:
df_combine.loc[df_combine['midup'] == True, 'ymid'] = 'Up'
df_combine.loc[df_combine['mideq'] == True, 'ymid'] = 'Equal'
df_combine.loc[df_combine['middown'] == True, 'ymid'] = 'Down'
df_combine['ymid'].value_counts()

In [ ]:
df_combine.loc[df_combine['spreadup'] == True, 'yspread'] = 'Up'
df_combine.loc[df_combine['spreadeq'] == True, 'yspread'] = 'Equal'
df_combine.loc[df_combine['spreaddown'] == True, 'yspread'] = 'Down'
df_combine['yspread'].value_counts()

In [ ]:
encoder = LabelEncoder()
df_combine['y'] = encoder.fit_transform(df_combine['ymid'])
X_train, X_test, y_train, y_test=train_test_split(df_combine[focus_columns], df_combine['y'], test_size=0.1 ,shuffle=False, stratify=None)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Part 5. Build Classification Models